(c) 2016 - present. Enplus Advisors, Inc.

This module uses:
* SP500 returns
* Security prices

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
sp5_df = pd.read_csv('sp500.csv', usecols=['date', 'adj_close'], parse_dates=['date'])
prices = pd.read_csv('prices.csv', parse_dates=['date'])

**Exercise:**

Merge the SP5 and security prices DataFrames, sorting by date ascending.

In [3]:
prc = pd.merge(sp5_df.rename(columns={'adj_close':'SP5'}), prices, on='date')
prc = prc.sort_values('date').reset_index(drop=True)

**Exercise:**

Calculate the beta of each stock to the SP500 using daily returns.

In [4]:
rtn1d = prc.set_index('date').pct_change().dropna()

In [5]:
def beta(y, X):
    mm = sm.add_constant(X)
    model = sm.OLS(y, mm).fit()
    return model.params[1]

def beta_df(df, y_col):
    
    y_idx = list(df.columns).index(y_col)
    x_cols = [(i, col) for i, col in enumerate(df.columns) if col != y_col]
    
    def calc_beta_df(X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        res = {}
        for i, col in x_cols:
            res[col] = beta(X[:, y_idx], X[:, i])
        return res
        
    return calc_beta_df

betas = beta_df(rtn1d, 'SP5')(rtn1d)

In [6]:
def beta_df_series(df, y):
    x_cols = [x for x in df.columns]
    
    def calc_beta_df(_df):
        res = {}
        xx, yy = _df.align(y, join='left', axis=0)
        for i, col in enumerate(x_cols):
            res[col] = beta(yy, xx.iloc[:, i])
        return res

def beta_df(df, y_col):
    
    y_idx = list(df.columns).index(y_col)
    x_cols = [(i, col) for i, col in enumerate(df.columns) if col != y_col]
    
    def calc_beta_df(X):
        res = {}
        for i, col in x_cols:
            res[col] = beta(X[:, y_idx], X[:, i])
        return res
        
    return calc_beta_df

In [7]:
beta_df(rtn1d, 'SP5')(rtn1d.values)

{'AAPL': 0.1879129168465805,
 'IBM': 0.4081097616078204,
 'MSFT': 0.38030797373942565}

**Exercise:**

Calculate the rolling 90-day beta for each stock.

In [8]:
def rolling_beta(y, X):
    xx, yy = X.align(y, join='left')
    return beta(yy, xx)

In [9]:
_calc_beta = beta_df(rtn1d, 'SP5')
grp = rtn1d.rolling(window=90,min_periods=90)